## Read in separate raw data files and create files of individual raster scans
### A. Ordog, June 2022
### NOTE: please do not modify
#### July 07, 2022: added in/out directories

## Import packages and define survey day and file directory

In [ ]:
import dva_sdhdf_combine_v3
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from astropy.modeling import models, fitting

day ='16'

#### Change the directory to where the files are located" ####
dir_in = '/home2/DATA_AO/DVA_DATA/survey_phase1_day'+day+'/'
dir_out = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
##############################################################

## Read in the file listing raster scan start and stop times:

In [ ]:
with open(dir_out+'dva_survey_phase1_day0'+day+'_raster1.txt') as fp:
    for line in fp:  
        raster1_start = line.split()[3]
        raster1_stop  = line.split()[4]
with open(dir_out+'dva_survey_phase1_day0'+day+'_raster2.txt') as fp:
    for line in fp:  
        raster2_start = line.split()[3]
        raster2_stop  = line.split()[4]

print(raster1_start)
print(raster1_stop)
print(raster2_start)
print(raster2_stop)


## Make files for each raster scan

### ***Note: skip this if files already made

In [ ]:
%%time

imp.reload(dva_sdhdf_combine_v3)

#outname = 'dva_survey_phase1_day0'+day+'_raster1'
#dva_sdhdf_combine_v3.combine(dir_in,dir_out,raster1_start,raster1_stop,outname,freq_s=2,freq_avg=True)

outname = 'dva_survey_phase1_day0'+day+'_raster2'
dva_sdhdf_combine_v3.combine(dir_in,dir_out,raster2_start,raster2_stop,outname,freq_s=2,freq_avg=True)

print('')

## Read in raster scan files:

In [ ]:
file = h5py.File(dir_out+'dva_survey_phase1_day0'+day+'_raster1'+'.h5','r')
freq_rast = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6
dataset1 = file['data']['beam_0']['band_SB0']['scan_0'] 
dec_rast1 = dataset1['metadata']['declination']
ra_rast1 = dataset1['metadata']['right_ascension']
el_rast1 = dataset1['metadata']['elevation']
az_rast1 = dataset1['metadata']['azimuth']
t_rast1 = dataset1['metadata']['utc']
noise_rast1 = dataset1['metadata']['noise_state']   
RR_rast1 = dataset1['data'][:,0,:]
LL_rast1 = dataset1['data'][:,1,:]
reRL_rast1 = dataset1['data'][:,2,:]
imRL_rast1 = dataset1['data'][:,3,:]

############################################################################
file = h5py.File(dir_out+'dva_survey_phase1_day0'+day+'_raster2'+'.h5','r')
dataset2 = file['data']['beam_0']['band_SB0']['scan_0']    
dec_rast2 = dataset2['metadata']['declination']
ra_rast2 = dataset2['metadata']['right_ascension']
el_rast2 = dataset2['metadata']['elevation']
az_rast2 = dataset2['metadata']['azimuth']
t_rast2 = dataset2['metadata']['utc']
noise_rast2 = dataset2['metadata']['noise_state']    
RR_rast2 = dataset2['data'][:,0,:]
LL_rast2 = dataset2['data'][:,1,:]
reRL_rast2 = dataset2['data'][:,2,:]
imRL_rast2 = dataset2['data'][:,3,:]

ra_rast1_deg = ra_rast1*360./24.
t_plt_rast1 = Time(t_rast1, format='isot',scale='utc').mjd

ra_rast2_deg = ra_rast2*360./24.
t_plt_rast2 = Time(t_rast2, format='isot',scale='utc').mjd

## Plot raster scans:

In [ ]:
# Formatting plot
fs = 12
fig1, axs = plt.subplots(1,2,figsize=(12,6))

freq_plt = 650. # in MHz
df = freq_rast[1]-freq_rast[0]
wf = np.where(abs(freq_rast-freq_plt)<df)[0][0]

RA1 = 12.5326*360/24
RA2 = 20.0042*360/24
DEC1 = 12.2674
DEC2 = 40.7963
t1_plt = Time(raster1_start, format='isot',scale='utc').mjd
t2_plt = Time(raster1_stop, format='isot',scale='utc').mjd
wplt1 = np.where((t_plt_rast1>=t1_plt) & (t_plt_rast1<=t2_plt))
t1_plt = Time(raster2_start, format='isot',scale='utc').mjd
t2_plt = Time(raster2_stop, format='isot',scale='utc').mjd
wplt2 = np.where((t_plt_rast2>=t1_plt) & (t_plt_rast2<=t2_plt))
dB1 = 68
dB2 = 74

# Just plotting LL total power for now - correcting for cos(dec):
plot1 = axs[0].scatter( (ra_rast1_deg[wplt1]-RA1)*np.cos(dec_rast1[wplt1]*np.pi/180.), dec_rast1[wplt1],
            s=20,c=10.*np.log10(LL_rast1[wplt1,wf]),cmap='viridis',vmin=72, vmax=73)
axs[0].scatter(0,DEC1,s=30,color='black')

# Just plotting LL total power for now - correcting for cos(dec):
plot2 = axs[1].scatter( (ra_rast2_deg[wplt2]-RA2)*np.cos(dec_rast2[wplt2]*np.pi/180.), dec_rast2[wplt2],
            s=20,c=10.*np.log10(LL_rast2[wplt2,wf]),cmap='viridis',vmin=74, vmax=77)
axs[1].scatter(0,DEC2,s=30,color='black')

for i in range(0,2):
    axs[i].set_xlabel('$\Delta$ RAcos($\delta$) (deg.)',fontsize=fs)
    axs[i].set_ylabel('Dec. (deg.)',fontsize=fs)
    axs[i].tick_params(axis="x", labelsize=fs)
    axs[i].tick_params(axis="y", labelsize=fs)
    axs[i].set_xlim(5,-5)
    axs[i].set(aspect='equal')
    axs[i].grid()
    #axs[i].set_title(str(src_name[source])+" Raster Scan, "+str(date))
    #fig.colorbar(plot, ax=axs)
#axs[0].set_ylim(DEC1-5,DEC1+5)
#axs[1].set_ylim(DEC2-5,DEC2+5)
plt.tight_layout()
#plt.savefig('../PLOTS/raster_with_noise_day2.jpg')

## Old code - Becky's work on Gaussian fits replaces this

In [ ]:
grid_x1, grid_y1 = np.mgrid[-5:5:1001j, DEC1-5:DEC1+5:1001j]
grid_z1 = griddata( ((ra_rast1_deg[wplt1]-RA1)*np.cos(dec_rast1[wplt1]*np.pi/180.), dec_rast1[wplt1]), 
                   LL_rast1[wplt1,wf][0], (grid_x1, grid_y1), method='linear')

grid_x2, grid_y2 = np.mgrid[-5:5:1001j, DEC2-5:DEC2+5:1001j]
grid_z2 = griddata( ((ra_rast2_deg[wplt2]-RA2)*np.cos(dec_rast2[wplt2]*np.pi/180.), dec_rast2[wplt2]), 
                   LL_rast2[wplt2,wf][0], (grid_x2, grid_y2), method='linear')

In [ ]:
# Formatting plot
fs = 12
fig1, axs = plt.subplots(1,2,figsize=(12,6))

# Just plotting LL total power for now - correcting for cos(dec):
plot1 = axs[0].imshow(grid_z1.T,vmin=1e6,vmax=2e7,zorder=0,extent=[-5,5,DEC1-5,DEC1+5],origin='lower')
axs[0].scatter(0,DEC1,s=30,color='black')

# Just plotting LL total power for now - correcting for cos(dec):
plot2 = axs[1].imshow(grid_z2.T,vmin=1e6,vmax=2e7,zorder=0,extent=[-5,5,DEC2-5,DEC2+5],origin='lower')
axs[1].scatter(0,DEC2,s=30,color='black')

for i in range(0,2):
    axs[i].set_xlabel('$\Delta$ RAcos($\delta$) (deg.)',fontsize=fs)
    axs[i].set_ylabel('Dec. (deg.)',fontsize=fs)
    axs[i].tick_params(axis="x", labelsize=fs)
    axs[i].tick_params(axis="y", labelsize=fs)
    axs[i].set_xlim(5,-5)
    axs[i].set(aspect='equal')
    axs[i].grid()
    #axs[i].set_title(str(src_name[source])+" Raster Scan, "+str(date))
    #fig.colorbar(plot, ax=axs)
axs[0].set_ylim(DEC1-5,DEC1+5)
axs[1].set_ylim(DEC2-5,DEC2+5)

In [ ]:
src_name = 'VIRA'
# provide an initial 'guess' at the model fit parameters
# Amplitude and the location of the y mean will differ for every source
if src_name == 'CYGA':
    amp = 5e8
    dec_mean = 41
if src_name == 'CASA':
    amp = 2.5e7
    dec_mean = 59
#**TAUA AND VIRA HAVE NOT BEEN OPTIMIZED
if src_name == 'TAUA':
    amp = 5e7
    dec_mean = 22
if src_name == 'VIRA':
    amp = 3e6
    dec_mean = 12.3
# (play around with how much you can vary these and still make the fit work)
# 2D Gaussian
g1 = models.Gaussian2D(amplitude=amp, x_mean=0., y_mean=dec_mean, x_stddev=1., y_stddev=1.) 
#g1 = models.Gaussian2D() 
# 2D constant offset value
g2 = models.Planar2D(slope_x = 0, slope_y = 0, intercept = 1e6) 
#g2 = models.Planar2D() 
    
# Add the two together
g_init = g2+g1
#print(g_init)

# The fitting algorithm (there may be other options):
fit_g = fitting.LevMarLSQFitter()

# The RA, DEC and LL data values to provide as x, y, z to the fitting algorithm:
x = (ra_rast2_deg[wplt2]-RA2)*np.cos(dec_rast2[wplt2]*np.pi/180.)
y = dec_rast2[wplt2]
z = LL_rast2[wplt2,wf]

print(x.shape)
print(y.shape)
print(z[0].shape)

# Do the fit:
# Inputs here are the initial guess, the x and y grid,
# and the z values of the Gaussian we set up:
fitfunc = fit_g(g_init, grid_x1, grid_y1, grid_z1)

#print(fitfunc)
print('Gaussian:')
print("Amplitude:", fitfunc[1].amplitude.value)
print("x mean:", fitfunc[1].x_mean.value)
print("y mean:", fitfunc[1].y_mean.value)
print("x standard deviation:", fitfunc[1].x_stddev.value)
print("y standard deviation:", fitfunc[1].y_stddev.value)
print("x fwhm:", fitfunc[1].x_fwhm)
print("y fwhm:", fitfunc[1].y_fwhm)
print('')
print('Offset:')
print("x slope:", fitfunc[0].slope_x.value)
print("y slope:", fitfunc[0].slope_y.value)
print("intercept:", fitfunc[0].intercept.value)